In [1]:
import linref as lr
import linref_011 as lr011
from linref.events import Rangel
from linref.events import common, selection, modify, utility, analyze
import numpy as np
import pandas as pd
import scipy.sparse as sparse
import geopandas as gpd
import linref as lr
import warnings

In [2]:
df_linear_1 = pd.DataFrame(index=[7, 6, 9, 4, 3, 2, 0, 1], data={
    'county': ['A', 'A', 'A', 'A', 'A' ,'B', 'B', 'B'],
    'route': ['Main St', 'Main St', 'Main St', '1st Ave', '1st Ave', '2nd Ave', '2nd Ave', 'Sycamore School Rd'],
    'beg': [0.0, 0.5, 1.1, 10.0, 12.0, 13.0, 13.1, 1.8],
    'end': [0.4, 1.1, 3.0, 12.0, 13.0, 13.1, 14.0, 3.0],
    'volume': [100, 200, 100, 400, 500, 150, 200, 800],
})
#df_linear_1 = pd.concat([df_linear_1] * 100, ignore_index=True)#.sort_values(['route', 'beg'])
df_linear_2 = pd.DataFrame({
    'route': ['Main St', 'Main St', '1st Ave', '2nd Ave', '2nd Ave', 'Sycamore School Rd'],
    'beg': [0.0, 1.0, 10.0, 13.0, 13.1, 2.2],
    'end': [1.0, 2.0, 12.0, 13.0, 14.0, 2.8],
    'class': ['A', 'B', 'A', 'A', 'B', 'A'],
})
#df_linear_2 = pd.concat([df_linear_2] * 100, ignore_index=True)#.sort_values(['route', 'beg'])

In [3]:
rng1 = Rangel(index=df_linear_1.index, groups=df_linear_1.route, begs=df_linear_1.beg, ends=df_linear_1.end, closed='left_mod')
rng2 = Rangel(index=df_linear_2.index, groups=df_linear_2.route, begs=df_linear_2.beg, ends=df_linear_2.end, closed='left_mod')
rng3 = Rangel(index=df_linear_1.index, groups=df_linear_1.route, locs=df_linear_1.beg)

In [4]:
c = Rangel(groups=['a', 'a', 'b', 'a', 'b', 'c'], begs=[0, 0, 2, 3, 4, 5], ends=[1, 2, 3, 4, 5, 6], closed='left_mod')
analyze.duplicated(c, subset=['begs'], keep='none')

array([ True,  True, False, False, False, False])

In [133]:
groups = np.array([['a', 'b'], ['b', 'c'], ['c', 'd'], ['a', 'd'], ['a', 'b'], ['c', 'd'], ['a', 'b'], ['b', 'c'], ['c', 'd'], ['a', 'd']])
tests = [('b', 'c'), ('a', 'c'), ('d', 'c'), ('a', 'e')]
groups = np.tile(groups, (1000, 1))
groups.shape

(10000, 2)

In [134]:
np.array([np.equal(groups, t).all(axis=1) for t in tests]).any(axis=0)

array([False,  True, False, ...,  True, False, False], shape=(10000,))

In [128]:
np.equal(groups, np.expand_dims(tests, 0))

ValueError: operands could not be broadcast together with shapes (10000,2) (1,2,4) 

In [135]:
np.equal([1,2,3], (1,2,3))

array([ True,  True,  True])

In [10]:
from linref.ext.base import LRS
lrs = LRS(['county', 'route'], 'loc', 'beg', 'end', 'geom', closed='left_mod')
df = pd.concat([df_linear_1] * 100000, ignore_index=True)
df.lr.set_lrs(lrs)
x = df.lr.shift(10)
x

,county,route,beg,end,volume
0,A,Main St,10.0,10.4,100
1,A,Main St,10.5,11.1,200
2,A,Main St,11.1,13.0,100
3,A,1st Ave,20.0,22.0,400
4,A,1st Ave,22.0,23.0,500
...,...,...,...,...,...
799995,A,1st Ave,20.0,22.0,400
799996,A,1st Ave,22.0,23.0,500
799997,B,2nd Ave,23.0,23.1,150
799998,B,2nd Ave,23.1,24.0,200


In [11]:
%%timeit
x = df.lr.shift(10)

86.6 ms ± 785 μs per loop (mean ± std. dev. of 7 runs, 10 loops each)


In [12]:
ec = lr011.EventsCollection(df, keys=['county', 'route'], beg='beg', end='end', closed='left_mod')

In [13]:
%%timeit
y = ec.shift(10)

53.9 ms ± 668 μs per loop (mean ± std. dev. of 7 runs, 10 loops each)
